In [311]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [319]:
data['campaign'] = data['campaign'].apply(lambda x: 0 if x == '(not set)' else 1)

### кол-во сессий у юзеров

In [322]:
count_user = pd.read_csv("tele2_dataset.csv", parse_dates=['date', 'event_time'], )
count_user = count_user[['client_id', 'session_id']].drop_duplicates().drop('session_id', axis=1)
count_user['sum'] = 1
count_user = count_user.groupby('client_id', as_index=False).agg('sum')

C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2323805512.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  count_user = pd.read_csv("tele2_dataset.csv", parse_dates=['date', 'event_time'], )


In [323]:
count_user['sum'] = 1
count_user = count_user.groupby('client_id', as_index=False).agg('sum')

In [324]:
count_user[count_user['sum'] >= 2]

,client_id,sum


In [325]:
count_user.reset_index(inplace=True)

In [326]:
data = data.merge(count_user, left_on = 'client_id', right_on='client_id')

In [327]:
data['sum'] = data['sum'].astype(int)
data['sum'] = data['sum'].apply(lambda x: 'Мало' if x <= 1 else 'Много')

### Проверка

In [331]:
data['campaign'] = data['campaign'].apply(lambda x: 1 if x != '(not set)' else 0)

In [386]:
data.head()

,date,campaign,source,medium,device,client_id,session_id,event_time,event_name,current_tariff,is_LSE_member,user_auth,user_type,region_name,age,gender,rfm_index,have_home_inet,index,sum
10234,2022-11-01,1,yandex.ru,referral,mobile,1965746663.166725,4525337134927472698,2022-10-31 21:02:20,1,Супер онлайн,0.0,1,B2C,largest_regions,25 - 39 лет,Male,132.0,0.0,14371,Мало
10782,2022-11-01,1,(direct),(none),mobile,822315664.166725,3975410978899350186,2022-10-31 21:07:04,1,Супер онлайн+,0.0,1,B2C,standart_regions,До 18 лет,Female,343.0,0.0,5587,Мало
10158,2022-11-01,1,google,organic,mobile,706554935.166725,4092338976582514873,2022-10-31 21:07:21,1,NaN,NaN,0,NaN,largest_regions,25 - 39 лет,Female,NaN,NaN,4782,Мало
10434,2022-11-01,1,(direct),(none),mobile,34173418.166725,7073798098886996667,2022-10-31 21:11:16,1,Федеральный S 09_2022,1.0,1,B2C,small_regions,25 - 39 лет,Male,231.0,0.0,212,Мало
9586,2022-11-01,1,(direct),(none),mobile,1296417728.166725,5312861579931468639,2022-10-31 21:15:42,1,Супер онлайн+ 04_2022,1.0,1,B2C,largest_regions,25 - 39 лет,Female,231.0,0.0,8913,Мало


In [385]:
data.event_name.unique()

array([1, 3, 2, 4, 5, 7, 6], dtype=int64)

In [368]:
data.campaign.value_counts()

campaign
1    172828
Name: count, dtype: int64

In [333]:
data['sum'].value_counts()

sum
Мало    174185
Name: count, dtype: int64

In [334]:
sorted_df = data[['client_id', 'session_id']].groupby('session_id', as_index=False).agg(lambda x: tuple(set(x.to_list())))
sorted_df = sorted_df[sorted_df['client_id'].apply(lambda x: len(x) > 1)]
sorted_df_list = sorted_df['session_id'].to_list()

data = data[data.session_id.apply(lambda x: x not in sorted_df_list)]

In [335]:
data = data.sort_values('event_time')
data['event_name'] = data.event_name.apply(lambda x: make_group(x))

In [336]:
viborka = data[(data['campaign'] == 1) & (data['gender'] == 'Female') & (data['region_name'] == 'largest_regions')]

In [337]:
viborka = do_voronka_uliana(viborka)
viborka

{1: {0: 2763, 2: 1711, 3: 1794, 4: 123, 5: 33, 6: 30, 7: 21},
 2: {0: 420, 1: 234, 3: 212, 4: 1767, 5: 26, 6: 0, 7: 9},
 3: {0: 717, 1: 501, 2: 870, 4: 8, 5: 0, 6: 0, 7: 0},
 4: {0: 198, 1: 161, 2: 18, 3: 26, 5: 1559, 6: 5, 7: 11},
 5: {0: 155, 1: 159, 2: 10, 3: 18, 4: 50, 6: 56, 7: 1175},
 6: {0: 17, 1: 15, 2: 0, 3: 0, 4: 4, 5: 5, 7: 51},
 7: {0: 1071, 1: 190, 2: 7, 3: 1, 4: 0, 5: 0, 6: 0}}

In [338]:
gen_s = do_voronka_uliana(data)
gen_s

{1: {0: 15737, 2: 10037, 3: 10793, 4: 653, 5: 172, 6: 142, 7: 132},
 2: {0: 1963, 1: 1159, 3: 1054, 4: 11484, 5: 213, 6: 3, 7: 70},
 3: {0: 3975, 1: 2799, 2: 5483, 4: 36, 5: 3, 6: 0, 7: 4},
 4: {0: 969, 1: 839, 2: 120, 3: 145, 5: 10418, 6: 13, 7: 80},
 5: {0: 792, 1: 991, 2: 61, 3: 115, 4: 324, 6: 243, 7: 8328},
 6: {0: 52, 1: 73, 2: 3, 3: 5, 4: 16, 5: 29, 7: 226},
 7: {0: 6954, 1: 1867, 2: 17, 3: 5, 4: 1, 5: 3, 6: 0}}

### работа с группами

In [339]:
for val in gen_s.values():
    sm = 0
    for el in val.values():
        sm += el
    for el in val:
        con = val[el]
        val[el] = val[el] / sm

{1: {0: 0.4178038549354856,
  2: 0.2664737428981044,
  3: 0.2865448945999044,
  4: 0.017336590028142092,
  5: 0.004566452503584134,
  6: 0.0037699782297031807,
  7: 0.003504486805076196},
 2: {0: 0.1231029725322965,
  1: 0.07268280446506961,
  3: 0.06609808102345416,
  4: 0.7201806095572557,
  5: 0.013357581838705631,
  6: 0.0001881349554747272,
  7: 0.004389815627743635},
 3: {0: 0.3231707317073171,
  1: 0.2275609756097561,
  2: 0.44577235772357726,
  4: 0.002926829268292683,
  5: 0.00024390243902439024,
  6: 0.0,
  7: 0.0003252032520325203},
 4: {0: 0.07700254291163382,
  1: 0.06667196439923713,
  2: 0.009535918626827717,
  3: 0.01152256834075016,
  5: 0.8278766687857597,
  6: 0.0010330578512396695,
  7: 0.006357279084551812},
 5: {0: 0.07296849087893864,
  1: 0.09130274553160125,
  2: 0.005620047908605122,
  3: 0.010595172286714576,
  4: 0.029850746268656716,
  6: 0.022388059701492536,
  7: 0.7672747374239911},
 6: {0: 0.12871287128712872,
  1: 0.1806930693069307,
  2: 0.00742574257

In [340]:
sex_ = ['Male', 'Female']
regs_ = ['largest_regions', 'big_regions', 'standart_regions', 'small_regions', 'mini_regions']
reclms = [0, 1]

In [397]:
print('Пол /', 'Группа регионов /', 'Есть ли реклама /', 'Размер выборки /', 'первое д', 'второе д', 'Значение к в группе /', 'Значение к в ген.сов /', 'z')
ans = []
for sex in sex_:
    for reg in regs_:
        for camp in reclms:
            viborka = data[(data['campaign'] == camp) & (data['gender'] == sex) & (data['region_name'] == reg)]
            viborka = do_voronka_uliana(viborka)
            for key in viborka:
                sm = 0
                for el in viborka[key]:
                    sm += viborka[key][el]
                if sm != 0:
                    for el in viborka[key]:
                        viborka[key][el] = viborka[key][el] / sm
                        vib_con = viborka[key][el]
                        gen_con = gen_s[key][el]
                        z = (vib_con - gen_con) / ((gen_con * (1 - gen_con) / sm) ** (1 / 2))
                        if z > 1.96 or z < -1.96:
                            if vib_con > 0 and vib_con > gen_con and el != 0:
                                ans += [[sex, reg, camp, sm, key, el, vib_con, gen_con, z]]

Пол / Группа регионов / Есть ли реклама / Размер выборки / первое д второе д Значение к в группе / Значение к в ген.сов / z


C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2677945507.py:17: RuntimeWarning: invalid value encountered in scalar divide
  z = (vib_con - gen_con) / ((gen_con * (1 - gen_con) / sm) ** (1 / 2))
C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2677945507.py:17: RuntimeWarning: invalid value encountered in scalar divide
  z = (vib_con - gen_con) / ((gen_con * (1 - gen_con) / sm) ** (1 / 2))
C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2677945507.py:17: RuntimeWarning: invalid value encountered in scalar divide
  z = (vib_con - gen_con) / ((gen_con * (1 - gen_con) / sm) ** (1 / 2))
C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2677945507.py:17: RuntimeWarning: invalid value encountered in scalar divide
  z = (vib_con - gen_con) / ((gen_con * (1 - gen_con) / sm) ** (1 / 2))
C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2677945507.py:17: RuntimeWarning: invalid value encountered in scalar divide
  z = (vib_con - gen_con) / ((gen_con * (1 - gen_con) / sm) ** (1 / 2))


In [403]:
sex_ = ['Male', 'Female']
regs_ = ['largest_regions', 'big_regions', 'standart_regions', 'small_regions', 'mini_regions']
reclms = [1]

In [382]:
ans.sort(key=lambda x: x[6], reverse=True)

In [383]:
print(*ans, sep='\n')

['Female', 'small_regions', 1, 1617, 4, 5, 0.8886827458256029, 0.8278766687857597, 6.477378900693271]
['Male', 'small_regions', 1, 1659, 4, 5, 0.8812537673297167, 0.8278766687857597, 5.759376404140455]
['Female', 'small_regions', 1, 1486, 5, 7, 0.8405114401076716, 0.7672747374239911, 6.680992335718325]
['Male', 'small_regions', 1, 1518, 5, 7, 0.813570487483531, 0.7672747374239911, 4.268544333112948]
['Female', 'small_regions', 1, 1869, 2, 4, 0.8004280363830926, 0.7201806095572557, 7.728160819820442]
['Female', 'big_regions', 1, 851, 5, 7, 0.7978848413631022, 0.7672747374239911, 2.1131599458614723]
['Male', 'small_regions', 1, 1907, 2, 4, 0.795490298898794, 0.7201806095572557, 7.325994486816101]
['Female', 'big_regions', 1, 1185, 2, 4, 0.7586497890295358, 0.7201806095572557, 2.9499352479300587]
['Male', 'big_regions', 1, 1053, 2, 4, 0.7502374169040835, 0.7201806095572557, 2.172688262241644]
['Female', 'big_regions', 1, 843, 3, 2, 0.5290628706998813, 0.44577235772357726, 4.86528925816868

In [434]:
print('Пол /', 'Группа регионов /', 'Есть ли реклама /', 'Размер выборки /', 'первое д', 'второе д', 'Значение к в группе /', 'Значение к в ген.сов /', 'z')
ans_2 = []
gen_c = (len(data[data['event_name'] == 7])) / (len(data[data['event_name'] == 1]) - 6700)
for sex in sex_:
    for reg in regs_:
        for camp in reclms:
            viborka = data[(data['campaign'] == camp) & (data['gender'] == sex) & (data['region_name'] == reg)]
            col_buy = len(viborka[viborka['event_name'] == 7])
            col_start = int(len(viborka[viborka['event_name'] == 1]) * 0.96)
            vib_c = col_buy / col_start
            z = (vib_c - gen_c) / ((gen_c * (1 - gen_c) / col_start) ** (1 / 2))
            if z > 1.96 or z < -1.96: 
                print(sex, reg, camp, col_start, vib_c, gen_c, z)
                if vib_c > gen_c:
                    ans_2 += [[sex, reg, camp, col_start, vib_c, gen_c, z]]

Пол / Группа регионов / Есть ли реклама / Размер выборки / первое д второе д Значение к в группе / Значение к в ген.сов / z


ZeroDivisionError: division by zero

In [420]:
ans_2.sort(key=lambda x: x[4], reverse=True)
print(*ans_2, sep='\n')

['Female', 'small_regions', 1, 5208, 0.24865591397849462, 0.19096952430285763, 10.591180588696151]
['Male', 'small_regions', 1, 5554, 0.23154483255311487, 0.19096952430285763, 7.69308076111026]
['Female', 'big_regions', 1, 3362, 0.21683521713265913, 0.19096952430285763, 3.815561837484853]


In [442]:
for_nelli = pd.read_csv('tele2_dataset.csv')
for_nelli.head()

C:\Users\Ульяна\AppData\Local\Temp\ipykernel_18116\2687037057.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for_nelli = pd.read_csv('tele2_dataset.csv')


,date,campaign,source,medium,device,client_id,session_id,event_time,event_name,current_tariff,is_LSE_member,user_auth,user_type,region_name,age,gender,rfm_index,have_home_inet
0,2022-11-01,eshop_art_alw_sim_none_yandex_search_brand-pro...,yandex,cpc,desktop,117251283.164232,5988994287612526860,2022-11-01 12:23:11,Покупка,Мой разговор,0.0,1,B2C,TYUMEN,25 - 39 лет,Male,141.0,0.0
1,2022-11-01,eshop_art_alw_sim_none_yandex_search_brand-pro...,yandex,cpc,desktop,117251283.164232,5988994287612526860,2022-11-01 12:22:49,Клик на кнопку Продолжить 3 (Экран выбора спос...,Мой разговор,0.0,1,B2C,TYUMEN,25 - 39 лет,Male,141.0,0.0
2,2022-11-01,eshop_art_alw_sim_none_yandex_search_brand-pro...,yandex,cpc,desktop,117251283.164232,5988994287612526860,2022-11-01 12:22:19,Ввод контактного номера,Мой разговор,0.0,1,B2C,TYUMEN,25 - 39 лет,Male,141.0,0.0
3,2022-11-01,eshop_art_alw_sim_none_yandex_search_brand-pro...,yandex,cpc,desktop,117251283.164232,5988994287612526860,2022-11-01 12:22:59,Клик на кнопку Продолжить 4 (Экран выбора адре...,Мой разговор,0.0,1,B2C,TYUMEN,25 - 39 лет,Male,141.0,0.0
4,2022-11-01,eshop_art_alw_sim_none_yandex_search_brand-pro...,yandex,cpc,desktop,117251283.164232,5988994287612526860,2022-11-01 12:22:21,Ввод имени,Мой разговор,0.0,1,B2C,TYUMEN,25 - 39 лет,Male,141.0,0.0


In [443]:
for_nelli.campaign.value_counts()

campaign
(not set)                                                                           141744
eshop_art_alw_sim_none_yandex_search_brand-common-tariffs_all-devices_none_rf         4251
eshop_art_alw_sim_none_yandex_search_brand-shop_all-devices_none_rf                   3259
4AQ1                                                                                  2600
mar_art_flight_flight-8_oct-2022_yandex_search_kpz_all-devices_none_regions           2073
                                                                                     ...  
mar_art_dop_ponedelniki_oct-2022_yandex_search_general_all-devices_none_vld              1
cbm_art_alw_premium_none_yandex_search_all_all-devices_none_rf                           1
mar_art_dop_ponedelniki_oct-2022_yandex_search_brand_all-devices_none_rnd                1
11_november_lineage                                                                      1
eshop_art_alw_sim_none_yandex_search_brand-common-delivery_all-devices_test-B_rf 

In [ ]:
'Female', 'small_regions', 1
'Male', 'small_regions', 1
'Female', 'big_regions', 1

In [449]:
def region_groups(x):
    if x in largest_regions:
        return 'largest_regions'
    elif x in big_regions:
        return 'big_regions'
    elif x in standart_regions:
        return 'standart_regions'
    elif x in small_regions:
        return 'small_regions'
    else:
        return 'mini_regions'
    

for_nelli['region_name'] = for_nelli['region_name'].apply(region_groups)

In [458]:
for_nelli[
    (for_nelli['gender'] == 'Female') & (for_nelli['region_name'] == 'small_regions') & (for_nelli['campaign'] != '(not set)')
][['client_id', 'event_name', 'event_time']].drop_duplicates().to_csv('for_nelli_conversions_1.csv')

In [459]:
for_nelli[
    (for_nelli['gender'] == 'Male') & (for_nelli['region_name'] == 'small_regions') & (for_nelli['campaign'] == '(not set)')
][['client_id', 'event_name', 'event_time']].drop_duplicates().to_csv('for_nelli_conversions_2.csv')

In [460]:
for_nelli[
    (for_nelli['gender'] == 'Female') & (for_nelli['region_name'] == 'big_regions') & (for_nelli['campaign'] == '(not set)')
][['client_id', 'event_name', 'event_time']].drop_duplicates().to_csv('for_nelli_conversions_3.csv')

In [461]:
for key in viborka:
    sm = 0
    for el in viborka[key]:
        sm += viborka[key][el]
    for el in viborka[key]:
        viborka[key][el] = viborka[key][el] / sm
        vib_con = viborka[key][el]
        gen_con = gen_s[key][el]
        z = (vib_con - gen_con) / (gen_con * (1 - gen_con) / sm) ** (1 / 2)
        if z > 1.96 or z < -1.96:
            print('Есть значимые различия', vib_con, gen_con, key, el)